In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import env
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer
import scipy
import sklearn.linear_model
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
import wrangle
import viz_explore
import scipy.stats as stats
import brian_model
import cluster_model

from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
import sklearn.preprocessing
import warnings

In [2]:
# MY PARTITIONS

df, train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, \
train_scaled, X_train_scaled, y_train_scaled, validate_scaled, X_validate_scaled, \
y_validate_scaled, test_scaled, X_test_scaled, y_test_scaled \
= wrangle.wrangle_zillow()

## Let's take a moment to run some feature engineering....

In [3]:
# This cell runs my function in feature_engineering.py that selects for the top 'k' number 
# of features
import brian_feature_engineering
f_feature = brian_feature_engineering.select_kbest(X_train_scaled,y_train_scaled, 5)
f_feature

['bedrooms', 'condition', 'rooms', 'tax_amount', 'price_per_sq_ft']

In [4]:
# Likewise, here we are using a function from the feature_engineering.py file to select for the best RFE features

rfe_feature = brian_feature_engineering.select_rfe(X_train_scaled,y_train_scaled, 5)
rfe_feature

['condition',
 'lot_size',
 'structure_value',
 'sq_ft_per_bathroom',
 'price_per_sq_ft']

#  Modeling

### Scaling
I have done modeling on the scaled datasets

### Baseline
I will use MEAN as the baseline

### Features
I will use ____ as my kbest features; _______ as my rfe features
All the X_train_scaled, X_validate_scaled, etc. are scaled from X_train, etc.¶
All of the X_train_kbest, X_train_rfe, etc are derived from the scaled datasets

In [5]:
# This cell is running a function in my brian_model.py file
# It creates data sets for modeling based on the KBest and RFE features, respectively:
X_train_kbest, X_validate_kbest, X_test_kbest, X_train_rfe, X_validate_rfe, X_test_rfe \
= brian_model.create_data_for_models(X_train_scaled, X_validate_scaled, X_test_scaled)

In [6]:
# 1. Predict logerror_pred_mean
# I create here new columns in the y_ data sets to hold the baseline value I am working with.
logerror_pred_mean = y_train_scaled.logerror.mean()
y_train_scaled['logerror_pred_mean'] = round(logerror_pred_mean, 2)
y_validate_scaled['logerror_pred_mean'] = round(logerror_pred_mean,2)
y_test_scaled['logerror_pred_mean'] = round(logerror_pred_mean,2)

In [7]:
y_train_scaled


,logerror,logerror_pred_mean
0,0.600496,0.6
1,0.606472,0.6
2,0.615869,0.6
3,0.599217,0.6
4,0.595534,0.6
...,...,...
22156,0.593047,0.6
22157,0.597474,0.6
22158,0.601797,0.6
22159,0.598466,0.6


In [8]:
# 2. RMSE of logerror_pred_mean
# Here, I calculate the Root Mean Squared Error of the baseline and print it
from sklearn.metrics import mean_squared_error

rmse_train = mean_squared_error(y_train_scaled.logerror,
                                y_train_scaled.logerror_pred_mean) ** .5
rmse_validate = mean_squared_error(y_validate_scaled.logerror, y_validate_scaled.logerror) ** (0.5)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

RMSE using Mean
Train/In-Sample:  0.02 
Validate/Out-of-Sample:  0.0


In [9]:
# Here, I create a DataFrame to insert the evaluation metrics of the different models; this datafram will be 
# taken in by the modeling function below and expanded on.
metric_df = pd.DataFrame(data=[
            {
                'model': 'mean_baseline', 
                'RMSE_train': rmse_train,
                'RMSE_validate': rmse_validate
                }
            ])
metric_df

,model,RMSE_train,RMSE_validate
0,mean_baseline,0.019087,0.0


In [10]:
def run_ols_model_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, metric_df):
    '''
    Function that runs the ols model on the kbest data
    
    '''
    from sklearn.metrics import mean_squared_error
    # create the model object
    lm = LinearRegression()
    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    lm.fit(X_train_kbest, y_train_scaled.logerror)
    # predict train
    y_train_scaled['logerror_pred_lm_kbest'] = lm.predict(X_train_kbest)
    # evaluate: rmse
    rmse_train = mean_squared_error(y_train_scaled.logerror, y_train_scaled.logerror_pred_lm_kbest) ** .5
    # predict validate
    y_validate_scaled['logerror_pred_lm_kbest'] = lm.predict(X_validate_kbest)
        # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate_scaled.logerror, y_validate_scaled.logerror_pred_lm_kbest) ** (0.5)
    
    metric_df = metric_df.append({
        'model': 'OLS Regressor KBEST', 
        'RMSE_train': rmse_train,
        'RMSE_validate': rmse_validate,
        }, ignore_index=True)

    # print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
    #     "\nValidation/Out-of-Sample: ", rmse_validate)

    return metric_df

In [11]:
run_ols_model_kbest(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, metric_df)

,model,RMSE_train,RMSE_validate
0,mean_baseline,0.019087,0.000000
1,OLS Regressor KBEST,0.018949,0.018989


In [12]:
metric_df

,model,RMSE_train,RMSE_validate
0,mean_baseline,0.019087,0.0


In [13]:
metric_df = brian_model.run_all_models_on_all_data(X_train_kbest, y_train_scaled, X_validate_kbest, y_validate_scaled, X_train_rfe, X_validate_rfe, X_test_kbest, X_test_rfe, metric_df)

In [14]:
metric_df

,model,RMSE_train,RMSE_validate
0,mean_baseline,0.019087,0.000000
1,OLS Regressor KBEST,0.018949,0.018989
2,OLS Regressor RFE,0.018944,0.018993
3,Lasso_alpha1_KBEST,0.018968,0.018994
4,Lasso_alpha1_RFE,0.018968,0.018994
5,glm_compound_kbest,0.018952,0.018989
6,glm_compound_rfe,0.018953,0.018993
7,quadratic_kbest,0.018929,0.018993
8,quadratic_rfe,0.018935,0.019002
